# 色彩空间

## 1 色彩空间转换  

在数字图像中，最常见的彩色模型是RGB模型（红、绿、蓝，在OpenCV中彩色图像组织的顺序是B-G-R，仍然是RGB模型。
这种模型是硬件处理的常用模型，比如采集图像的CCD传感器、显示图像的显示器等等。

符合描述人类眼睛观察的则是HSV（色度、饱和度、亮度）模型。

OpenCV中有274种颜色空间转换方法。我们只研究两种最广泛使用的转换方法，BGR↔Gray和BGR↔HSV。   

In [1]:
import cv2
flags=[i for i in dir(cv2) if i.startswith('COLOR_')] 
len(flags)

346

### 1.1 转换GRAY色彩空间

GRAY色彩空间：GRAY（灰度图像）通常指8位灰度图，其具有256个灰度级，像素值的范围是[0,255]。

- 当图像由RGB色彩空间转换为GRAY色彩空间时，其处理方式如下：

    Gray=0.299*R+0.587*G+0.114*B    
    
    
对于颜色转换，使用函数cv.cvtColor(input_image，flag)，其中flag确定转换类型。 
- 对于BGR→Gray转换，我们使用标志cv.COLOR_BGR2GRAY。
- 类似地，对于BGR→HSV，我们使用标志cv.COLOR_BGR2HSV

In [ ]:
import cv2
img_src = cv2.imread("./images/lena.jpg")
img_gray = cv2.cvtColor(img_src,cv2.COLOR_BGR2GRAY)    #彩色转灰度
print(img_gray.shape)    #单通道
img_color = cv2.cvtColor(img_gray,cv2.COLOR_GRAY2BGR)  #灰度转彩色
print(img_color.shape)   #3通道 
cv2.imshow('img_src',img_src)
cv2.imshow('img_color',img_color)
cv2.imshow('img_gray',img_gray)
cv2.waitKey()
cv2.destroyAllWindows()  

在彩色图像转换到灰度图像后，

再用该灰度图转换回彩色图像只是名义上多通道的彩色图像，人眼看到的却不是“彩色”了。 

applyColorMap()，它可以将灰度图像转换成彩色图像，转换的模式达22种之多。

In [17]:
import cv2 
img_gray = cv2.imread("./images/lena.jpg",0)
for i in range(22):
    dst = cv2.applyColorMap(img_gray,i) 
    cv2.imshow('map',dst) 
    cv2.waitKey(500)
   # cv2.imwrite("map-"+str(i)+".jpg",dst) 
cv2.destroyAllWindows()  

### 1.2 转换到HSV空间  

HSV色彩空间

- RGB是从硬件的角度提出的颜色模型，在与人眼匹配的过程中可能存在一定的差异，HSV色彩空间是一种面向视觉感知的颜色模型

- 指出人眼的色彩知觉主要包含三要素：色调（Hue，也称为色相）、饱和度（Saturation）、亮度（Value）。

- 色调指光的颜色，饱和度是指色彩的深浅程度，亮度指人眼感受到的光的明暗程度。


In [5]:
import cv2

img = cv2.imread("./images/lena.jpg")
hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
cv2.imshow("rgb", img)
cv2.imshow("hsv", hsv)
cv2.waitKey(0)
cv2.destroyAllWindows()


根据色调的值获取某种颜色，也就是可以通过在HSV的H通道上的值

In [13]:
img = cv2.imread("./images/lena.jpg")
hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
img[:, :, 0] = 255
Blue = img
blueHSV = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
cv2.imshow("imgBlue", Blue)
cv2.imshow("blueHSV", blueHSV)
cv2.waitKey()
cv2.destroyAllWindows()


### 1.3 HSV空间-物体跟踪 

通过cv2.inRange()可以将图像内指定范围的值标注出来，在返回到mask中。

如果图像的值位于该区间，则mask对应位置上的值为255，反之为0。

通过掩摸按位与运算将指定颜色取出来。

bitwise_and有了第3个参数mask，利用掩摸（mask）进行“与”操作

即掩膜图像白色区域是对需要处理图像像素的保留，黑色区域是对需要处理图像像素的剔除。

In [6]:
import cv2
import numpy as np

img = cv2.imread("./images/pinpan.jpg")
hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
cv2.imshow("hsv", hsv)

# 定蓝色的阈值
# lower = np.array([110, 50, 50])
# upper = np.array([130, 255, 255])

#黄色-乒乓球
lower = np.array([15, 100, 100])
upper = np.array([30, 255, 255]) 

# 换到 HSV
hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
# 根据阈值构建掩模
mask = cv2.inRange(hsv, lower, upper)
# 对原图像和掩模位运算
res = cv2.bitwise_and(img, img, mask=mask)

cv2.imshow("yellow", res)
cv2.waitKey()
cv2.destroyAllWindows()

通过HSV实现在摄像头的物体跟踪

In [3]:
import cv2
import numpy as np

'''
物体跟踪

• 从视频中获取每一帧图像
• 将图像转换到 HSV 空间
• 设置 HSV 阈值到蓝色范围。
'''

cap = cv2.VideoCapture(0)
ret = cap.set(3, 640)
ret = cap.set(4, 480)

# 定蓝色的阈值
# lower = np.array([110, 50, 50])
# upper = np.array([130, 255, 255])

#黄色-乒乓球
lower = np.array([20, 100, 100])
upper = np.array([30, 255, 255])

# 黑色
# lower_black = np.array([0, 0, 0])
# upper_black = np.array([180, 255, 30])

while True:
    # 获取每一帧
    ret, frame = cap.read()
    # 换到 HSV
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # 根据阈值构建掩模
    mask = cv2.inRange(hsv, lower, upper)
    # mask = cv2.inRange(hsv, lower_black, upper_black)
    # 对原图像和掩模位运算
    res = cv2.bitwise_and(frame, frame, mask=mask)

    # 显示图像
    cv2.imshow('frame', frame)
    cv2.moveWindow('frame', x=0, y=0)  # 原地
    cv2.imshow('mask', mask)
    cv2.moveWindow('mask', x=frame.shape[1], y=0)#右边
    cv2.imshow('res', res)
    cv2.moveWindow('res', y=frame.shape[0], x=0)#下边

    k = cv2.waitKey(1)  # & 0xFF
    if k == ord('q'):
        break
# 关闭窗口
cap.release()
cv2.destroyAllWindows()


### 1.4 查找确定HSV值 

这是我们在stackoverflow.com中常见的问题。其实解决这个问题非常简单，你可以使用相同的函数cv.cvtColor()。

你只需传递所需的BGR值，而不是传递图像。例如，要查找绿色的HSV值，在Python终端中输入以下命令：


In [4]:
import cv2
import numpy as np

# wrong
# green=np.uint8([0,255,0])
# hsv_green=cv2.cvtColor(green,cv2.COLOR_BGR2HSV)
# hsv_green 

# 三层括号应 分别对应于 cvArray cvMat IplImage
green = np.uint8([[[0, 255, 0]]])
hsv_green = cv2.cvtColor(green, cv2.COLOR_BGR2HSV)
hsv_green

array([[[ 60, 255, 255]]], dtype=uint8)

现在分别将[H-10,100,100]和[H+10,255,255]作为下限和上限。

除了这种方法，你可以使用任何图像编辑工具如GIMP，或任何在线转换器来查找这些值，但不要忘记调整HSV范围。